In [1]:
%matplotlib inline
import pandas as pd
import mpld3
import folium
import nivapy

# Initial NivaPy demo

Currently two sub-modules:

 * **nivapy.da:** Functions for data access
 
 * **nivapy.spatial:** Basic mapping
 
Note: None of this will be very useful until all this data is sitting in a single relational database. At the moment, it only queries RESA2, not NIVADATABASE (but these should be merged anyway).

## 1. Connect to db

In [2]:
# Connect to db
engine = nivapy.da.connect()

## 2. Search projects

In [3]:
# Search projects
prj_df = nivapy.da.search_projects("project_name LIKE 'ICPW_TOCTRENDS_2015%'", engine)
prj_df

,project_id,project_number,project_name,contact_person,project_description
0,3930,None,ICPW_TOCTRENDS_2015_SE_RIVER,None,None
1,3810,None,ICPW_TOCTRENDS_2015_FI,None,None
2,3870,None,ICPW_TOCTRENDS_2015_US_LTM,None,None
3,3890,None,ICPW_TOCTRENDS_2015_Cz2,None,None
4,3891,None,ICPW_TOCTRENDS_2015_US_TIME,None,None
5,3912,None,ICPW_TOCTRENDS_2015_CA_QU,None,None
6,4010,None,ICPW_TOCTRENDS_2015_SE_EXCLUDED,None,None
7,4011,None,ICPW_TOCTRENDS_2015_SE_RIVER_EXCLUDED,None,None
8,4012,None,ICPW_TOCTRENDS_2015_NO,None,None
9,4150,None,ICPW_TOCTRENDS_2015_US_LTM_EXCLUDED,None,None


## 3. Search for stations within projects

In [4]:
# Get stations for NO, SE and FL
stn_df = nivapy.da.select_project_stations([4012, 3830, 3810], engine)

print 'Number of stations:', len(stn_df)
stn_df.head()

Number of stations: 370


,station_id,station_code,station_name,latitude,longitude,altitude
0,110,1845-601,Tennvatn,67.593796,15.487890,339
1,112,1101-43,Glypstadvatnet,58.487223,6.188384,252
2,115,831-501,BrÃ¥rvatn,59.294921,7.727118,902
3,116,1640-603,Tufsingen,62.612294,11.876558,781
4,118,1034-19,Indre Espelandsvatnet,58.305398,7.160170,391


## 3. Basic map

In [5]:
# Simple map
map1 = nivapy.spatial.quickmap(stn_df)
map1

## 5. Extract water chemistry

Hover the mouse over the plots to display **pan and zoom tools** in the bottom-left corner of the plot area.

In [6]:
# Station ID and parameters of interest
stn_id = 29617
par_list = ['pH', 'KOND', 'TOC', 'TOTN', 'TOTP', 'Cr']

# Get data between 2010 and 2015
wc_df, dup_df, fig = nivapy.da.extract_water_chem(stn_id, 
                                                  par_list, 
                                                  '2010-01-01', 
                                                  '2015-12-31',
                                                  engine, 
                                                  plot=True)

# Plot
mpld3.display()

In [7]:
wc_df.head()

,Cr_µg/l,KOND_mS/m,TOC_mg C/l,TOTN_µg/l N,TOTP_µg/l P,pH_None,Cr_flag,KOND_flag,TOC_flag,TOTN_flag,TOTP_flag,pH_flag
sample_date,,,,,,,,,,,,
2010-01-04,0.20,4.57,3.5,535.0,5.0,6.98,None,None,None,None,None,None
2010-02-08,0.34,4.97,3.1,585.0,6.0,7.06,None,None,None,None,None,None
2010-03-08,0.30,4.79,3.0,560.0,5.0,7.23,None,None,None,None,None,None
2010-04-06,1.70,6.25,4.6,900.0,31.0,7.04,None,None,None,None,None,None
2010-05-10,2.30,4.73,6.0,610.0,17.0,7.16,None,None,None,None,None,None


In [8]:
# Resample to annual averages
wc_df.resample('A').mean()

,Cr_µg/l,KOND_mS/m,TOC_mg C/l,TOTN_µg/l N,TOTP_µg/l P,pH_None
sample_date,,,,,,
2010-12-31,0.523125,4.508125,4.54375,545.312500,13.625000,7.128750
2011-12-31,0.571250,4.555000,4.41125,590.625000,21.437500,7.046250
2012-12-31,0.411250,4.478750,4.37500,578.750000,16.125000,7.094375
2013-12-31,0.485556,4.652222,3.90000,559.166667,19.888889,7.195556
2014-12-31,0.502143,4.390625,4.15000,520.312500,18.812500,7.117500
2015-12-31,0.327750,4.648125,4.53125,605.937500,19.312500,7.210000


In [9]:
# More complex data manipulation e.g. do a calculation, resample
# to an unusual frequency (e.g. 2-monthly) and return the std. dev.
(wc_df*5).resample('2M').std().head()

,Cr_µg/l,KOND_mS/m,TOC_mg C/l,TOTN_µg/l N,TOTP_µg/l P,pH_None
sample_date,,,,,,
2010-01-31,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-31,0.141421,0.636396,0.353553,88.388348,3.535534,0.601041
2010-05-31,4.678996,5.541029,3.718198,903.742220,48.862051,0.347491
2010-07-31,1.284848,0.534634,1.892969,447.446459,10.408330,0.390512
2010-09-30,0.106066,0.565685,7.071068,0.000000,0.000000,0.848528


## 6. Extract area-scaled discharge

The plot can be panned and zoomed, as above.

In [10]:
# Get flow data between 2010 and 2015 for the same WC site
# Automatically scales flows based on ratio of catchment areas
q_df, fig = nivapy.da.extract_discharge(stn_id, 
                                        '2010-01-01', 
                                        '2015-12-31',
                                        engine,
                                        plot=True)

# Plot
mpld3.display()

## 7. Convert co-ordinates

### 7.1. Decimal degrees to UTM

In [11]:
# Convert dd to UTM zone 33
stn_df = nivapy.spatial.wgs84_dd_to_utm(stn_df)

stn_df.head()

,station_id,station_code,station_name,latitude,longitude,altitude,utm_north,utm_east,utm_zone
0,110,1845-601,Tennvatn,67.593796,15.487890,339,7.497659e+06,520752.998531,33
1,112,1101-43,Glypstadvatnet,58.487223,6.188384,252,6.516678e+06,-12829.988260,33
2,115,831-501,BrÃ¥rvatn,59.294921,7.727118,902,6.595516e+06,86269.002041,33
3,116,1640-603,Tufsingen,62.612294,11.876558,781,6.946273e+06,339741.000001,33
4,118,1034-19,Indre Espelandsvatnet,58.305398,7.160170,391,6.489485e+06,41191.004036,33


### 7.2. UTM to WGS84 decimal degrees

In [12]:
# Convert back to dd. The new cols, 'lat' and 'lon', should
# equal the originals, 'latitude' and 'longitude'
stn_df = nivapy.spatial.utm_to_wgs84_dd(stn_df)

stn_df.head()

,station_id,station_code,station_name,latitude,longitude,altitude,utm_north,utm_east,utm_zone,lat,lon
0,110,1845-601,Tennvatn,67.593796,15.487890,339,7.497659e+06,520752.998531,33,67.593796,15.487890
1,112,1101-43,Glypstadvatnet,58.487223,6.188384,252,6.516678e+06,-12829.988260,33,58.487223,6.188384
2,115,831-501,BrÃ¥rvatn,59.294921,7.727118,902,6.595516e+06,86269.002041,33,59.294921,7.727118
3,116,1640-603,Tufsingen,62.612294,11.876558,781,6.946273e+06,339741.000001,33,62.612294,11.876558
4,118,1034-19,Indre Espelandsvatnet,58.305398,7.160170,391,6.489485e+06,41191.004036,33,58.305398,7.160170


## 8. Plot shapefile

Very large shapefiles will crash Folium, but nevertheless useful for quick visualisations of simple shapefiles. Automatically re-projects from the data co-ordinates to WGS84 geographic co-ordinates.

In [13]:
# Plot shapefile
in_shp = (r'C:\Data\James_Work\Useful_Stuff\GIS_Data\Norway\Shapefiles\NO_Fylker_pol.shp')
map2 = nivapy.spatial.plot_shapefile(in_shp)

map2